In [1]:
import os
import cv2
import numpy as np
import numpy as np
import pandas as pd
from scipy.fft import fft2, ifft2, ifftshift, fftshift
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def ResizeWithAspectRatio(image, width=None, height=None, inter=cv2.INTER_AREA):
    dim = None
    (h, w) = image.shape[:2]

    if width is None and height is None:
        return image
    if width is None:
        r = height / float(h)
        dim = (int(w * r), height)
    else:
        r = width / float(w)
        dim = (width, int(h * r))

    return cv2.resize(image, dim, interpolation=inter)

In [ ]:
img = cv2.imread("Crunch.jpg")

resize = ResizeWithAspectRatio(img, width=1280) # Resize by width OR
cv2.imshow('rfc crunch', resize)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
pix_width = resize.shape[0]
pix_height = resize.shape[1]
color_channel_cnt = resize.shape[2]
pix_width

In [ ]:
def smear(phi = 0):
    img = cv2.imread("Crunch.jpg")
    altered_img = img

    for r in range(0, pix_width):
        for c in range(0, pix_height):
            for ch in range(0, color_channel_cnt):
                try:
                    theta = np.tanh(c / r) + phi
                except:
                    theta = phi
                radius = np.sqrt((r**2 + c**2))
                altered_img[r][c][ch] = img[min(abs(int(radius * np.cos(theta))), 2999)][min(abs(int(radius * np.sin(theta))), 2999)][ch]
                
    resize = ResizeWithAspectRatio(altered_img, width=1280)

    return resize




In [ ]:
def smear_2(opening_ratio = 1, img = resize):
    granularity = 1
    x_max = pix_width / 2
    # opening_ratio = 1/5  # tau / T


    x = np.arange(-x_max, x_max, granularity)
    X, Y = np.meshgrid(x, x)

    sinc = (1 / (X * np.pi * opening_ratio)) * np.sin(np.pi * X * opening_ratio) * (1 / (Y * np.pi * opening_ratio)) * np.sin(np.pi * Y * opening_ratio)
    sinc_n = sinc / (150*np.nanmax(sinc))
    sinc_rgb = np.dstack((sinc_n, sinc_n, sinc_n))
    return sinc_rgb * img

In [ ]:
for i in range(100, 1000):
    imgw = smear_2(1/i)
    cv2.imwrite(f'gens_3/{i}.jpg', resize * imgw)


In [ ]:
import time
img_array = []
for i in range(100, 500):
    sinc = smear_2(1/i)
    time.sleep(50)
    cv2.imwrite(f'gens_3/{i}.jpg', sinc)
#     img_array.append(sinc)
#     height, width, layers = sinc.shape
#     size = (width,height)

# out = cv2.VideoWriter(f'rfc.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 10, size)

# for i in range(len(img_array)):
#     out.write(img_array[i])
# out.release()


In [ ]:
img_cnt = 100
for n in range(0, img_cnt):
    phi = img_cnt / 2 * np.sin(2 * np.pi * n / img_cnt) + img_cnt / 2
    img = smear(phi = phi)
    path = f'C:/Users/nicor/.vscode/Digital_Audio/Fiesta_Red/gens_2/{n}.jpg'
    cv2.imwrite(filename = path, img = img)
    

In [ ]:
import os
import cv2

img_array = []
file_array = []
base = 'C:/Users/nicor/.vscode/Digital_Audio/Fiesta_Red/gens_3/'
frame_cnt = len([entry for entry in os.listdir(base) if os.path.isfile(os.path.join(base, entry))])


for i in range(100, 1000):
    file_array.append(base + str(i) + '.jpg')

for i in range(0, len(file_array)):
    img = cv2.imread(file_array[i], 1)
    img_array.append(img * i)
    height, width, layers = img.shape
    size = (width,height)

out = cv2.VideoWriter(f'rfc2.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 100, size)

for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

In [ ]:
granularity = 0.01
x_max = 5
tau_1 = 1
T_1 = 1

x = np.arange(-x_max, x_max, granularity)
X, Y = np.meshgrid(x, x)

r1 = 0

for n in range(-x_max, x_max):
    try:
        an = (1/(n * np.pi) * np.sin((n * np.pi * tau_1) / (T_1))) 
    except:
        an = tau_1 / T_1

    r1 = r1 + an * np.cos((2 * np.pi * n * X) / T_1)
print(type(r1))
t_r1 = np.tile(r1, (3, 3))
print(len(t_r1))
t_r1_norm = t_r1 / np.linalg.norm(t_r1, ord = 2, axis = 0, keepdims = True) - 0.01
rgb_r1_t = np.dstack((t_r1_norm, t_r1_norm, t_r1_norm))
plt.imshow(rgb_r1_t * img)

In [ ]:
def sinc_gen(opening_ratio):   
    granularity = 0.01
    x_max = 5
    # opening_ratio = 1/5  # tau / T


    x = np.arange(-x_max, x_max, granularity)
    X, Y = np.meshgrid(x, x)

    sinc = (1 / (X * np.pi * opening_ratio)) * np.sin(np.pi * X * opening_ratio) + np.sin(np.pi * Y * opening_ratio)
    

    t_r1 = np.tile(sinc, (3, 3))
    print(t_r1.max())
    t_r1_norm = t_r1 / t_r1.max()
    rgb_r1_t = np.dstack((t_r1_norm, t_r1_norm, t_r1_norm))

    return rgb_r1_t *img / 255

In [ ]:
img.max()

In [ ]:
a = sinc_gen(1/10)
plt.set_cmap('viridis')
plt.imshow(np.clip(a, 0, 1))

In [ ]:
for n in np.arange(1, 200):
    plt.imsave(f'gens/{n}.jpg', arr = np.clip(sinc_gen(opening_ratio = n), 0, 1))